In [4]:
!pip install upstox


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for upstox: filename=upstox-2.0.3-py2.py3-none-any.whl size=12548 sha256=9b8aa5a60f3dbb521ef27e4d0e490e7ecaa528c0de4350ca3c61f8c73b89e01f
  Stored in directory: c:\users\vishn\appdata\local\pip\cache\wheels\15\09\f0\49be9532ebe08d1152ae1e79188247d6017a2e5d8bbb0e55df
Successfully built upstox


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [4]:
import requests
import pandas as pd 

df = pd.read_csv('complete.csv')
total_df = pd.read_csv('ind_nifty100list.csv')
symbols = total_df['Symbol'].tolist()

# looping over symbols and getting there corresponding instrument key
# from df
is_key = {}
count = 0
for symbol in symbols:
    try:
        is_key[symbol] = df[(df['tradingsymbol'] == symbol) & (df['instrument_type'] == 'EQUITY')]['instrument_key'].values[0]
        count += 1
    except:
        print(symbol)
print(count)
print(is_key)

101
{'ABB': 'NSE_EQ|INE117A01022', 'ADANIENSOL': 'NSE_EQ|INE931S01010', 'ADANIENT': 'NSE_EQ|INE423A01024', 'ADANIGREEN': 'NSE_EQ|INE364U01010', 'ADANIPORTS': 'NSE_EQ|INE742F01042', 'ADANIPOWER': 'NSE_EQ|INE814H01011', 'ATGL': 'NSE_EQ|INE399L01023', 'AMBUJACEM': 'NSE_EQ|INE079A01024', 'APOLLOHOSP': 'NSE_EQ|INE437A01024', 'ASIANPAINT': 'NSE_EQ|INE021A01026', 'DMART': 'NSE_EQ|INE192R01011', 'AXISBANK': 'NSE_EQ|INE238A01034', 'BAJAJ-AUTO': 'NSE_EQ|INE917I01010', 'BAJFINANCE': 'NSE_EQ|INE296A01024', 'BAJAJFINSV': 'NSE_EQ|INE918I01026', 'BAJAJHLDNG': 'NSE_EQ|INE118A01012', 'BANKBARODA': 'NSE_EQ|INE028A01039', 'BERGEPAINT': 'NSE_EQ|INE463A01038', 'BEL': 'NSE_EQ|INE263A01024', 'BPCL': 'NSE_EQ|INE029A01011', 'BHARTIARTL': 'NSE_EQ|INE397D01024', 'BOSCHLTD': 'NSE_EQ|INE323A01026', 'BRITANNIA': 'NSE_EQ|INE216A01030', 'CANBK': 'NSE_EQ|INE476A01014', 'CHOLAFIN': 'NSE_EQ|INE121A08PJ0', 'CIPLA': 'NSE_EQ|INE059A01026', 'COALINDIA': 'NSE_EQ|INE522F01014', 'COLPAL': 'NSE_EQ|INE259A01022', 'DLF': 'NSE_EQ|

In [2]:
# import json
# # Assuming you have a JSON string stored in a variable called 'json_str'
# formatted_json = json.dumps(json.loads(response.text), indent=4)
# data = json.loads(response.text)

print(data['data']['candles'])


NameError: name 'data' is not defined

In [5]:
import json
def get_list_of_candles(symbol, start_date, end_date, interval):
    url = f"https://api.upstox.com/v2/historical-candle/{is_key[symbol]}/{interval}/{end_date}/{start_date}"

    payload={}
    headers = {
    'Accept': 'application/json'
    }

    response = requests.request("GET", url, headers=headers, data=payload)
    data = json.loads(response.text)
    candles = []
    for candle in data['data']['candles']:
        candles.append(candle)
    return candles

In [12]:
from datetime import datetime
def create_sql_statement(symbol):
    sql_stat = 'INSERT IGNORE INTO stock_price (Symbol,Timestamp,Open,Close,High,Low) VALUES \n'
    candles = get_list_of_candles(symbol, '2024-04-04', '2024-04-10', '1minute')
    candles.extend(get_list_of_candles(symbol, '2024-03-05', '2024-04-02', '1minute'))
    candles.extend(get_list_of_candles(symbol, '2023-09-05', '2024-03-03', '30minute'))
    candles.extend(get_list_of_candles(symbol, '2018-09-05', '2023-09-04', 'day'))
    t = set()
    for candle in candles:
        time_stamp = candle[0]
        dt = datetime.strptime(time_stamp, '%Y-%m-%dT%H:%M:%S%z')
        # Convert to SQL format
        sql_timestamp = dt.strftime('%Y-%m-%d %H:%M:%S')

        t.add(f"('{symbol}','{sql_timestamp}',{candle[1]},{candle[4]},{candle[2]},{candle[3]}),\n")
    sql_stat += ''.join(list(t))
    sql_stat = sql_stat[:-2] + ';'

    print(len(candles))
    return sql_stat

create_sql_statement('RELIANCE')

11168


"INSERT IGNORE INTO stock_price (Symbol,Timestamp,Open,Close,High,Low) VALUES \n('RELIANCE','2024-03-05 15:08:00',3000.65,2999.7,3000.75,2999),\n('RELIANCE','2024-03-18 12:03:00',2841.9,2843.8,2844.15,2841.1),\n('RELIANCE','2024-03-13 15:18:00',2868.25,2867.95,2868.35,2867.85),\n('RELIANCE','2022-09-12 00:00:00',2570.55,2598.05,2612,2570.55),\n('RELIANCE','2024-03-27 12:26:00',2978.9,2981,2981.65,2978.5),\n('RELIANCE','2024-03-27 11:02:00',2964.1,2964.05,2965.15,2963.2),\n('RELIANCE','2024-03-15 11:24:00',2852.95,2853,2853,2852.3),\n('RELIANCE','2024-03-11 12:06:00',2954.15,2954,2955,2953.2),\n('RELIANCE','2024-03-12 11:12:00',2960.1,2960,2960.85,2958.65),\n('RELIANCE','2024-03-06 12:35:00',2963.85,2965,2965,2963.7),\n('RELIANCE','2024-03-20 11:05:00',2864.9,2864.75,2865.6,2863.9),\n('RELIANCE','2024-03-26 11:30:00',2888.15,2888,2888.4,2887.15),\n('RELIANCE','2024-04-02 15:13:00',2974.85,2974.9,2975,2974.55),\n('RELIANCE','2024-04-01 11:59:00',2973.4,2974.55,2974.85,2973.25),\n('RELIAN

In [14]:
!pip install pymysql
import pymysql

connection = pymysql.connect(
    host='localhost',
    user='root',
    password='toor',
    database='paper_trade'
)

# Create a cursor object to execute SQL queries
cursor = connection.cursor()

# Execute SQL queries
# ...
for symbol in symbols:
    sql_statement = create_sql_statement(symbol)
    if symbol not in sql_statement:
        print(symbol, 'not found')
        continue
    print(symbol)
    cursor.execute(sql_statement)
    connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


DEPRECATION: torchsde 0.2.5 has a non-standard dependency specifier numpy>=1.19.*; python_version >= "3.7". pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of torchsde or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


11168
ABB
11168
ADANIENSOL
11168
ADANIENT
11168
ADANIGREEN
11168
ADANIPORTS
11168
ADANIPOWER
11127
ATGL
11168
AMBUJACEM
11168
APOLLOHOSP
11168
ASIANPAINT
11168
DMART
11168
AXISBANK
11168
BAJAJ-AUTO
11168
BAJFINANCE
11168
BAJAJFINSV
11168
BAJAJHLDNG
11168
BANKBARODA
11168
BERGEPAINT
11168
BEL
11168
BPCL
11168
BHARTIARTL
11168
BOSCHLTD
11168
BRITANNIA
11168
CANBK
0
CHOLAFIN not found
11168
CIPLA
11168
COALINDIA
11168
COLPAL
11168
DLF
11168
DABUR
11168
DIVISLAB
11168
DRREDDY
11168
EICHERMOT
11168
GAIL
11168
GODREJCP
11168
GRASIM
11168
HCLTECH
11168
HDFCBANK
11168
HDFCLIFE
11168
HAVELLS
11168
HEROMOTOCO
11168
HINDALCO
11164
HAL
11168
HINDUNILVR
11168
ICICIBANK
11168
ICICIGI
11168
ICICIPRULI
11168
ITC
11168
IOC
10900
IRCTC
10575
IRFC
11168
INDUSINDBK
11168
NAUKRI
11168
INFY
11168
INDIGO
11168
JSWSTEEL
11168
JINDALSTEL
9942
JIOFIN
11168
KOTAKBANK
11168
LTIM
11168
LT
10256
LICI
11168
M&M
11168
MARICO
11168
MARUTI
11168
NTPC
10516
NESTLEIND
11168
ONGC
11168
PIDILITIND
11168
PFC
11168
POWERGRID

In [15]:
with open('insert_stock_price.sql', 'w') as f:
    for symbol in symbols:
        sql_statement = create_sql_statement(symbol)
        if symbol not in sql_statement:
            print(symbol, 'not found')
            continue
        f.write(sql_statement + '\n')

11168
11168
11168
11168
11168
11168
11127
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
0
CHOLAFIN not found
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11164
11168
11168
11168
11168
11168
11168
10900
10575
11168
11168
11168
11168
11168
11168
9942
11168
11168
11168
10256
11168
11168
11168
11168
10516
11168
11168
11168
11168
11168
11168
11168
10794
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
11167
11168
11168
11168
11168
11168
11168
11168
11168
11168
11168
10789
11168
11168
10457
11168
